In [ ]:
# Validate the sa_fast_v2 output
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.strtree import STRtree
from shapely.ops import unary_union
from decimal import Decimal
import json

# Tree geometry
def get_tree_polygon():
    trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
    base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
    tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
    trunk_bottom_y = -trunk_h
    vertices = [
        (float(Decimal('0.0')), float(tip_y)),
        (float(top_w / Decimal('2')), float(tier_1_y)),
        (float(top_w / Decimal('4')), float(tier_1_y)),
        (float(mid_w / Decimal('2')), float(tier_2_y)),
        (float(mid_w / Decimal('4')), float(tier_2_y)),
        (float(base_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(base_y)),
        (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
        (float(-(trunk_w / Decimal('2'))), float(base_y)),
        (float(-(base_w / Decimal('2'))), float(base_y)),
        (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
        (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
        (float(-(top_w / Decimal('4'))), float(tier_1_y)),
        (float(-(top_w / Decimal('2'))), float(tier_1_y)),
    ]
    return Polygon(vertices)

TREE_POLY = get_tree_polygon()

def parse_s_value(s_val):
    if isinstance(s_val, str) and s_val.startswith('s'):
        return float(s_val[1:])
    return float(s_val)

def load_submission(path):
    df = pd.read_csv(path)
    df['x_val'] = df['x'].apply(parse_s_value)
    df['y_val'] = df['y'].apply(parse_s_value)
    df['deg_val'] = df['deg'].apply(parse_s_value)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    return df

def create_tree(x, y, deg):
    return translate(rotate(TREE_POLY, deg, origin=(0, 0)), x, y)

def get_bbox_side(polygons):
    if not polygons:
        return 0
    combined = unary_union(polygons)
    bounds = combined.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def check_overlaps(polygons, tolerance=1e-12):
    if len(polygons) <= 1:
        return []
    overlaps = []
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx > i:
                if polygons[i].intersects(polygons[idx]) and not polygons[i].touches(polygons[idx]):
                    intersection = polygons[i].intersection(polygons[idx])
                    if intersection.area > tolerance:
                        overlaps.append((i, idx, intersection.area))
    return overlaps

# Load and validate
df = load_submission('/home/code/experiments/001_bbox3_optimization/submission2.csv')
print(f"Loaded {len(df)} rows")

scores_by_n = {}
overlap_configs = []

for n in range(1, 201):
    n_df = df[df['n'] == n]
    if len(n_df) != n:
        continue
    polygons = [create_tree(row['x_val'], row['y_val'], row['deg_val']) for _, row in n_df.iterrows()]
    overlaps = check_overlaps(polygons)
    if overlaps:
        overlap_configs.append((n, len(overlaps)))
    side = get_bbox_side(polygons)
    scores_by_n[n] = (side ** 2) / n

total_score = sum(scores_by_n.values())
print(f"\nTotal score: {total_score:.6f}")
print(f"Baseline: 70.647327")
print(f"Improvement: {70.647327 - total_score:.6f}")
print(f"Overlaps: {len(overlap_configs)}")
if overlap_configs:
    print(f"Overlap configs: {overlap_configs[:10]}")

In [ ]:
# Save metrics
metrics = {
    'cv_score': total_score,
    'baseline': 70.647327,
    'improvement': 70.647327 - total_score,
    'overlap_configs': len(overlap_configs),
    'target': 68.888293,
    'gap_to_target': total_score - 68.888293
}

with open('/home/code/experiments/001_bbox3_optimization/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics: {metrics}")

# If improved and no overlaps, copy to submission
if total_score < 70.647327 and len(overlap_configs) == 0:
    import shutil
    shutil.copy('/home/code/experiments/001_bbox3_optimization/submission2.csv', '/home/submission/submission.csv')
    print("Copied improved submission!")
else:
    print("No improvement or has overlaps - keeping baseline")